In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

#from psycopg import Cursor
#from psycopg.sql import SQL, Identifier
#from psycopg_pool import ConnectionPool
from semantic_kernel.connectors.memory.postgres.postgres_memory_store import (
    PostgresMemoryStore,
)

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
endpoint = os.getenv("GPT_OPENAI_ENDPOINT")
api_key = os.getenv("GPT_OPENAI_KEY")
gpt_deployment_name = os.getenv("GPT_OPENAI_DEPLOYMENT_NAME")
conn_str = os.getenv("PG_CONN_STR_PY")
ada_deployment_name = "ada"

In [3]:
kernel = sk.Kernel()
azure_chat_service = AzureChatCompletion(deployment_name=gpt_deployment_name, endpoint=endpoint, api_key=api_key)
azure_text_embedding = AzureTextEmbedding(deployment_name=ada_deployment_name, endpoint=endpoint, api_key=api_key)
kernel.add_chat_service("chat_completion", azure_chat_service)
kernel.add_text_embedding_generation_service("ada", azure_text_embedding)

mem_store = PostgresMemoryStore(conn_str,1536,1,3)
kernel.register_memory_store(memory_store=mem_store)
kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "text_memory")

KernelPlugin(name='text_memory', description=None, functions={'recall': KernelFunction(plugin_name='text_memory', description='Recall a fact from the long term memory', name='recall', is_semantic=False, stream_function=<bound method TextMemoryPlugin.recall of TextMemoryPlugin()>, parameters=[ParameterView(name='input', description='The information to retrieve', default_value='', type_='string', required=False), ParameterView(name='limit', description='The maximum number of relevant memories to recall.', default_value='1', type_='string', required=False), ParameterView(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value='0.75', type_='string', required=False), ParameterView(name='collection', description='The collection to search for information', default_value='generic', type_='string', required=False)], delegate_type=<DelegateTypes.InStringAndContextOutTaskString: 13>, function=<bound method TextMemoryPlugin.recall of TextMemory

In [4]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information(collection="aboutMe", id="info1", text="My name is Andrea")
    await kernel.memory.save_information(collection="aboutMe", id="info2", text="I currently work as a tour guide")
    await kernel.memory.save_information(
        collection="aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information(
        collection="aboutMe",
        id="info4",
        text="I visited France and Italy five times since 2015",
    )
    await kernel.memory.save_information(collection="aboutMe", id="info5", text="My family is from New York")

In [5]:
await populate_memory(kernel)

In [6]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search("aboutMe", question)
        print(f"Answer: {result[0].text}\n")

In [7]:
await search_memory_examples(kernel)

Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: I've been living in Seattle since 2005

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I visited France and Italy five times since 2015

Question: what do I do for work
Answer: I currently work as a tour guide

